In [1]:
#Ref: Langchain in your pocket, Mehul Gupta
### !pip install pyowm langchain-experimental==0.0.46 langchain==0.0.349
import sys
import os
sys.path.append('/home/vino/api_keys')
from api_key import OPENAI_API_KEY, OWM_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

import warnings
warnings.filterwarnings('ignore')

In [2]:
#callback handler
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [3]:
handler = StdOutCallbackHandler()
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",temperature=0)
prompt = PromptTemplate.from_template("What is the capital of {country}?")
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(country='South Africa',callbacks=[handler])



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of South Africa?

> Finished chain.


'\n\nThe capital of South Africa is Pretoria.'

In [5]:
#without callback handler
chain.run(country='South Africa')

'\n\nThe capital of South Africa is Pretoria.'

In [6]:
#logger
from langchain.callbacks import FileCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from loguru import logger

In [7]:
logfile = "output.log"
logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",temperature=0)
prompt=PromptTemplate.from_template("What is the capital of {country}? ")
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
answer = chain.run(country='Argentina')
logger.info(answer)

2024-10-15 16:57:24.070 | INFO     | __main__:<module>:8 - 

The capital of Argentina is Buenos Aires.


In [10]:
#custom callback
from langchain.callbacks.base import BaseCallbackHandler

class MyHandler(BaseCallbackHandler):

    """Base callback handler that can be used to handle callbacks from langchain."""


    def on_llm_start(self,serialized:'Dict[str, Any]',prompts: 'List[str]', *, run_id:'UUID',parent_run_id:'Optional[UUID]'= None, tags: 'Optional[List[str]]' = None, metadata: 'Optional[Dict[str, Any]]' = None, **kwargs: 'Any'): 
        print('We are starting of with this prompt -->',prompts)
 
    def on_llm_end(self,response:'LLMResult', *, run_id: 'UUID', parent_run_id: 'Optional[UUID]' = None, **kwargs: 'Any'): 
        print('The LLM is done',response)

handler = MyHandler()
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",temperature=0)
prompt = PromptTemplate.from_template("What is the capital of {country}?")

chain = LLMChain(llm=llm, prompt=prompt)
chain.run(country='South Africa',callbacks=[handler])

We are starting of with this prompt --> ['What is the capital of South Africa?']
The LLM is done generations=[[Generation(text='\n\nThe capital of South Africa is Pretoria.', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens': 10}, 'model_name': 'gpt-3.5-turbo-instruct'} run=None


'\n\nThe capital of South Africa is Pretoria.'